# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
resposta = requests.get(url)

print("Chaves principais do JSON:")
print(resposta.json().keys())
print()

dados = resposta.json()['items']
df_plv = pd.DataFrame(dados)
print("🧾 Primeiras linhas do DataFrame:")
df_plv.head()

Chaves principais do JSON:
dict_keys(['items', 'hasMore', 'limit', 'offset', 'count', 'links'])

🧾 Primeiras linhas do DataFrame:


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,9252,Município,Maruim,2804003,SE,None,Arquivado,17944.000938/2008-38,2008-06-25T00:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Caixa Econômica Federal,Real,518100.0,0,0,29/09/2008
1,9253,Município,Santa Cruz da Baixa Verde,2612471,PE,None,Arquivado por decurso de prazo,17944.000938/2009-19,2011-01-10T00:00:00Z,Operação contratual interna,Provias 2009,Instituição Financeira Nacional,Banco do Brasil S/A,Real,200000.0,0,0,27/08/2012
2,9259,Município,Itamarandiba,3132503,MG,None,Deferido,17944.000939/2012-69,2012-06-22T00:00:00Z,Operação contratual interna,Provias 2012,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1250000.0,1,0,06/07/2012
3,9271,Município,Alto Paraíso,4128625,PR,None,Deferido,17944.000943/2013-16,2013-09-03T00:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Paraná S/A,Real,400000.0,1,0,08/10/2013
4,9273,Município,Bugre,3109253,MG,None,Deferido,17944.000944/2010-18,2010-06-29T00:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,306879.0,1,0,23/07/2010


In [3]:
# 2) Seu código aqui
df_plv['status'].value_counts()

status
Deferido                                                 2900
Deferido (PVL-IF)                                         715
Arquivado                                                 462
Arquivado por decurso de prazo                            291
Encaminhado à PGFN com manifestação técnica favorável     230
Regularizado                                              203
Arquivado a pedido                                         97
Em retificação pelo interessado                            23
Indeferido                                                 18
Em retificação pelo interessado (PVL-IF)                   11
Em retificação pelo credor (PVL-IF)                         8
Em retificação pelo credor                                  6
Arquivado a pedido (PVL-IF)                                 6
Cancelado                                                   6
Em análise                                                  5
Arquivado por decurso de prazo (PVL-IF)                     5
P

In [4]:
# 3) Seu código aqui
print(df_plv['data_status'].dtype)

ano_status = df_plv['data_status'].str[-4:].astype(int)
print(ano_status.head())

object
0    2008
1    2012
2    2012
3    2013
4    2010
Name: data_status, dtype: int64


In [5]:
# 4) Seu código aqui
frequencia_ano = ano_status.value_counts()
print(frequencia_ano)

data_status
2008    485
2010    346
2007    330
2014    298
2006    281
2012    277
2011    250
2013    249
2019    233
2016    221
2009    216
2004    200
2023    193
2022    175
2015    174
2018    157
2017    149
2021    143
2002    133
2020    125
2003    114
2024    113
2025     81
2005     57
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [6]:
#1) Seu código aqui
def consultar_solicitacoes(uf, tipo_interessado):
    url_base = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
    parametros = {
        'uf': uf,
        'tipo_interessado': tipo_interessado
    }
    resposta = requests.get(url_base, params=parametros)
    
    # Verifica se deu certo
    if resposta.status_code != 200:
        print(f"Erro na consulta: status code {resposta.status_code}")
        return None
    
    dados = resposta.json()['items']
    df = pd.DataFrame(dados)
    return df

In [7]:
# 2) Seu código aqui
df_mg_estado = consultar_solicitacoes('MG', 'Estado')

# Filtra pelo status
df_filtrado = df_mg_estado[df_mg_estado['status'] == 'Arquivado por decurso de prazo']

# Conta quantas linhas existem
quantidade = len(df_filtrado)
print(f"Quantidade de solicitações arquivadas por decurso de prazo em MG: {quantidade}")

Quantidade de solicitações arquivadas por decurso de prazo em MG: 1


In [8]:
# 3) Seu código aqui
df_ba_municipio = consultar_solicitacoes('BA', 'Município')
print(df_ba_municipio.columns)

Index(['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf',
       'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao',
       'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor',
       'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status'],
      dtype='object')


##### Comentario do Aluno
Podemos ver que não temos uma coluna referente aos nomes dos municípios, mas sim ao seus respectivos codigos do IBGE.
Então precisamos usa-los para ter uma resposta de qual município com mais solicitações, mas ao fazê-lo a resposta viria também com o digo e não o nome do municipio, o que faria necessario o uso da tabela propria do IBGE para sabermos de que municipio se trata.
Dessa forma achei de bom uso, trazer para o codigo a tabela do IBGE e fazer uso dela para melhorar o resultado

In [21]:
# Carrega os municípios direto do GitHub
url = 'https://raw.githubusercontent.com/leogermani/estados-e-municipios-ibge/master/municipios.csv'
df_municipios = pd.read_csv(url)

# Visualiza as colunas
print(df_municipios.columns)
print(df_municipios.head())

Index(['COD UF', 'COD', 'NOME'], dtype='object')
   COD UF      COD                   NOME
0      11  1100015  Alta Floresta D'oeste
1      11  1100023              Ariquemes
2      11  1100031                 Cabixi
3      11  1100049                 Cacoal
4      11  1100056             Cerejeiras
Index(['COD UF', 'COD', 'NOME'], dtype='object')


In [22]:
print(df_municipios.columns)

Index(['COD UF', 'COD', 'NOME'], dtype='object')


In [15]:

df_deferidos = df_ba_municipio[df_ba_municipio['status'] == 'Deferido']

contagem_municipios = df_deferidos['cod_ibge'].value_counts()

municipio_mais_deferidos = contagem_municipios.idxmax()
quantidade = contagem_municipios.max()

In [24]:
df_municipios.rename(columns={
    'COD': 'cod_ibge',
    'NOME': 'municipio'
}, inplace=True)

df_municipios['cod_ibge'] = df_municipios['cod_ibge'].astype(int)

In [25]:
df_com_nomes = df_deferidos.merge(
    df_municipios[['cod_ibge', 'municipio']],
    on='cod_ibge',
    how='left'
)

# Conta por município
contagem_municipios = df_com_nomes['municipio'].value_counts()

municipio_mais_deferidos = contagem_municipios.idxmax()
quantidade = contagem_municipios.max()

print(f"Município da Bahia com mais solicitações deferidas: {municipio_mais_deferidos} ({quantidade} solicitações)")

Município da Bahia com mais solicitações deferidas: Luís Eduardo Magalhães (16 solicitações)


In [26]:
# 4) Seu código aqui
df_ba_estado = consultar_solicitacoes('BA', 'Estado')

# Salva em CSV
df_ba_estado.to_csv('solicitacoes_bahia_estado.csv', index=False)

print("Arquivo 'solicitacoes_bahia_estado.csv' salvo com sucesso.")

Arquivo 'solicitacoes_bahia_estado.csv' salvo com sucesso.
